In [8]:
#install.packages("data.table","MASS","dunn.test")
library(data.table)
library(MASS)
library(dunn.test)

data = fread('workout_logs.csv', header = T, sep = ',', stringsAsFactors = T)
data$bmicat = as.factor(data$bmicat)
head(data[,3:4])

workout_logs,bmicat
5,2
3,2
9,2
8,2
3,2
4,2


We assume that the number of workouts follows Poisson distribution and depends on the BMI category (fixed effect), but also each individual has their own personalized intercept (random effect). This is a so-called [mixed-effect model](https://en.wikipedia.org/wiki/Mixed_model), super popular in stats whenever we have multiple observations per subject.

In [31]:
PQL <- glmmPQL(workout_logs ~ bmicat, ~ 1 | common_user_id, family = gaussian(),
               data = data[sample(nrow(data))[1:1000000],], verbose = TRUE)
summary(PQL)

iteration 1
iteration 2


Linear mixed-effects model fit by maximum likelihood
 Data: data[sample(nrow(data))[1:1e+06], ] 
  AIC BIC logLik
   NA  NA     NA

Random effects:
 Formula: ~1 | common_user_id
        (Intercept) Residual
StdDev:    7.273949 8.784084

Variance function:
 Structure: fixed weights
 Formula: ~invwt 
Fixed effects: workout_logs ~ bmicat 
                Value Std.Error     DF   t-value p-value
(Intercept) 11.049447 0.1734170 654969  63.71603       0
bmicat2     -1.594296 0.1747853 654969  -9.12145       0
bmicat3     -2.021963 0.1747686 654969 -11.56938       0
bmicat4     -2.218196 0.1751245 654969 -12.66640       0
 Correlation: 
        (Intr) bmict2 bmict3
bmicat2 -0.992              
bmicat3 -0.992  0.984       
bmicat4 -0.990  0.982  0.983

Standardized Within-Group Residuals:
        Min          Q1         Med          Q3         Max 
-16.2380261  -0.4070557  -0.2044558   0.1773150  62.1221324 

Number of Observations: 1000000
Number of Groups: 654973 

Here we can conclude that effects of BMI categories are significant and of the order of $-2$. For simply comparing the means you can also use Kruskal-Wallis test (instead of ANOVA, since the data is not normally distributed). While the result is very close to yours, it's slightly more accurate from stats perspective.

In [32]:
kruskal.testet.data = data[sample(nrow(data))[1:100000],]
kruskal.test(subset.data$workout_logs ~ subset.data$bmicat)


	Kruskal-Wallis rank sum test

data:  subset.data$workout_logs by subset.data$bmicat
Kruskal-Wallis chi-squared = 46.753, df = 3, p-value = 3.922e-10


This test only tells us if there is a difference between the groups. To check where the difference is there is a post-hoc Dunn test.

In [33]:
dunn.test(subset.data$workout_logs,subset.data$bmicat)

  Kruskal-Wallis rank sum test

data: x and group
Kruskal-Wallis chi-squared = 46.7534, df = 3, p-value = 0


                           Comparison of x by group                            
                                (No adjustment)                                
Col Mean-|
Row Mean |          1          2          3
---------+---------------------------------
       2 |   2.723396
         |    0.0032*
         |
       3 |   3.053436   1.891160
         |    0.0011*     0.0293
         |
       4 |   3.837929   5.975016   4.230552
         |    0.0001*    0.0000*    0.0000*

alpha = 0.05
Reject Ho if p <= alpha/2


Kruskal Wallis and Dunn don't take into account that we have multiple observations per subject so to be super correct you may need to first average all observation for each subject, or take a random observation from each subject, and use every subject only once.

All this might be a small overkill given the size of the dataset, but for the paper you will still need to report p-values even if they don't make much sense (since they are obviously almost zero due to the size of the data).